In [ ]:
# Note: This notebook will NOT run in the Project Road Trip environment (prt_env_yaml).
# In order to run this notebook you will need to install bs4.
# (But the sqlite database object that this notebook created is inlcuded in the repo!)

In [1]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import sqlite3

In [2]:
content = rq.get("http://users.econ.umn.edu/~holmes/data/BORDLIST.html").content

In [3]:
soup = BeautifulSoup(content)

In [4]:
table = soup.find('table')

In [5]:
# table

In [6]:
clean = []
for row in table.find_all('tr'):
    values = row.find_all('p')
    clean.append([value.text for value in values])

In [7]:
# clean

In [8]:
df = pd.DataFrame(clean[1:], columns=clean[0])

In [9]:
df.head()

,BORDINDX,ST1ST2,LENGTH
0,1,AL-FL,196.1
1,2,AL-GA,291.3
2,3,AL-MS,323.9
3,4,AL-TN,146.7
4,5,AR-LA,166.5


In [10]:
df = df.drop('BORDINDX', axis=1)

In [11]:
df['LENGTH'] = df['LENGTH'].map(float)

In [12]:
df[(df['ST1ST2'] == 'AZ-CO') | (df['ST1ST2'] == 'NM-UT') | (df['ST1ST2'] == 'DC-MD') | (df['ST1ST2'] == 'DC-VA')]

,ST1ST2,LENGTH
11,AZ-CO,0.0
26,DC-MD,24.6
27,DC-VA,1.9
95,NM-UT,0.0


In [13]:
df = df.drop([11, 26, 27, 95])

In [14]:
df = df.reset_index(drop=True)

In [15]:
df['Interstate'] = True

In [16]:
def no_interstates(border):
    if border in {'AR-MS', 'CO-OK', 'GA-NC', 'ID-NV', 'ID-WY', 'IA-WI', 'KS-NE', 'KY-MO',
                 'KY-VA', 'MI-WI', 'MO-NE', 'MT-SD', 'NE-SD', 'NV-OR', 'NM-OK', 'NY-VT'}:
        return False
    else:
        return True

In [17]:
df['Interstate'] = df['ST1ST2'].map(no_interstates)

In [18]:
df['TimeZone'] = [
    'Central', 'Central-Eastern', 'Central', 'Central',
    'Central', 'Central', 'Central', 'Central', 'Central', 'Central',
    'Mountain-Pacific', 'Mountain', 'Mountain-Pacific', 'Mountain',
    'Pacific', 'Pacific',
    'Mountain & Central-Mountain', 'Mountain', 'Mountain', 'Central-Mountain', 'Mountain', 'Mountain',
    'Eastern', 'Eastern', 'Eastern',
    'Eastern', 'Eastern', 'Eastern',
    'Eastern',
    'Eastern', 'Eastern', 'Eastern',
    'Central', 'Central', 'Central', 'Central', 'Central',
    'Mountain & Mountain-Pacific', 'Mountain-Pacific', 'Mountain & Pacific & Mountain-Pacific', 'Mountain',
        'Pacific', 'Mountain',
    'Central', 'Central & Central-Eastern', 'Central', 'Central', 'Central',
    'Central & Eastern', 'Eastern & Central-Eastern', 'Eastern',
    'Central', 'Central & Central-Mountain', 'Central',
    'Central', 'Eastern', 'Central & Eastern', 'Eastern', 'Eastern',
    'Central', 'Central',
    'Eastern', 'Eastern', 'Eastern', 'Eastern',
    'Eastern', 'Eastern', 'Eastern',
    'Eastern',
    'Eastern', 'Central',
    'Central', 'Central', 'Central',
    'Central', 'Central', 'Central',
    'Central',
    'Mountain & Central-Mountain', 'Mountain', 'Mountain',
    'Eastern', 'Eastern', 'Eastern',
    'Central & Mountain',
    'Central & Mountain', 'Mountain',
    'Eastern',
    'Eastern', 'Eastern',
    'Central-Mountain', 'Mountain & Central-Mountain',
    'Pacific', 'Mountain-Pacific',
    'Eastern', 'Eastern',
    'Eastern', 'Eastern',
    'Central',
    'Pacific',
    'Eastern',
    'Mountain',
    'Eastern',
    'Mountain',
    'Eastern'
                 ]

In [19]:
df.sample(20)

,ST1ST2,LENGTH,Interstate,TimeZone
1,AL-GA,291.3,True,Central-Eastern
90,NM-OK,34.5,False,Central-Mountain
51,KS-MO,307.5,True,Central
58,KY-WV,79.1,True,Eastern
55,KY-OH,155.5,True,Eastern
96,OH-PA,92.4,True,Eastern
47,IL-WI,144.4,True,Central
33,IA-MO,235.0,True,Central
93,NV-UT,344.6,True,Mountain-Pacific
50,IN-OH,178.8,True,Eastern


In [20]:
length_series = df['LENGTH'].sort_values(ascending=False).reset_index(drop=True)

In [21]:
df['LengthRank'] = 0

In [22]:
mapper = dict(zip(length_series, length_series.index + 1))

In [23]:
df['LengthRank'] = df['LENGTH'].map(mapper)

In [24]:
df.sample(20)

,ST1ST2,LENGTH,Interstate,TimeZone,LengthRank
14,CA-NV,608.2,True,Pacific,2
63,MA-RI,62.6,True,Eastern,94
1,AL-GA,291.3,True,Central-Eastern,27
42,ID-WY,170.7,False,Mountain,59
34,IA-NE,148.9,True,Central,67
5,AR-MO,325.3,True,Central,19
96,OH-PA,92.4,True,Eastern,81
32,IA-MN,262.1,True,Central,32
0,AL-FL,196.1,True,Central,50
73,MN-WI,291.1,True,Central,28


In [25]:
df[(df['LengthRank'] == 1) | (df['LengthRank'] == 105)]

,ST1ST2,LENGTH,Interstate,TimeZone,LengthRank
26,DE-NJ,1.4,True,Eastern,105
98,OK-TX,715.4,True,Central,1


In [26]:
df = df[['ST1ST2', 'LENGTH', 'LengthRank', 'Interstate', 'TimeZone']]

In [27]:
df.head(10)

,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone
0,AL-FL,196.1,50,True,Central
1,AL-GA,291.3,27,True,Central-Eastern
2,AL-MS,323.9,20,True,Central
3,AL-TN,146.7,68,True,Central
4,AR-LA,166.5,60,True,Central
5,AR-MO,325.3,19,True,Central
6,AR-MS,282.8,29,False,Central
7,AR-OK,198.2,49,True,Central
8,AR-TN,125.1,72,True,Central
9,AR-TX,65.0,93,True,Central


In [28]:
river_content = rq.get('https://en.wikipedia.org/wiki/List_of_river_borders_of_U.S._states').content
river_soup = BeautifulSoup(river_content)

In [29]:
river_list = river_soup.find('div', id='mw-content-text').find_all('ul')[1]

In [30]:
river_list.find_all('li')[0].text

'Arthur Kill: New Jersey, New York (tidal strait)'

In [31]:
rivers = {}

for bord in river_list.find_all('li'):
    colon = bord.text.index(':')
    rivers[bord.text[:colon]] = bord.text[colon+2:]

In [32]:
rivers

{'Arthur Kill': 'New Jersey, New York (tidal strait)',
 'Big Sandy River': 'Kentucky, West Virginia',
 'Big Sioux River': 'South Dakota, Iowa',
 'Blackwater River': 'Virginia, North Carolina',
 'Bois de Sioux River': 'South Dakota, Minnesota, North Dakota',
 'Brule River': 'Michigan, Wisconsin',
 'Byram River': 'Connecticut, New York',
 'Catawba River': 'North Carolina, South Carolina',
 'Chattahoochee River': 'Alabama, Florida, Georgia',
 'Chattooga River': ' Georgia, South Carolina',
 'Colorado River': 'Arizona, Nevada, California, Baja California',
 'Columbia River': 'Washington, Oregon',
 'Connecticut River': 'New Hampshire, Vermont',
 'Delaware River': 'New York, Pennsylvania, New Jersey, Delaware',
 'Des Moines River': 'Iowa, Missouri',
 'Detroit River': 'Michigan, Ontario',
 'Great Miami River (mouth only)': 'Ohio, Indiana',
 'Halls Stream': 'New Hampshire, Quebec',
 'Hudson River (lower part only)': 'New Jersey, New York',
 'Kill Van Kull': 'New Jersey, New York (tidal strait)'

In [33]:
len(rivers.values()) == len(set(rivers.values()))

False

In [34]:
revers = {}
revers['AL-FL'] = 'Perdido'
revers['AL-GA'] = 'Chattahoochee'
revers['AL-MS'] = 'Tennessee'
revers['AR-MO'] = 'St. Francis'
revers['AR-MS'] = 'Mississippi'
revers['AR-OK'] = 'Poteau'
revers['AR-TN'] = 'Mississippi'
revers['AR-TX'] = 'Red (South)'
revers['AZ-CA'] = 'Colorado'
revers['AZ-NV'] = 'Colorado'
revers['CT-NY'] = 'Byram'
revers['CT-RI'] = 'Pawcatuck'
revers['DE-NJ'] = 'Delaware'
revers['FL-GA'] = 'St. Marys'
revers['GA-SC'] = 'Savannah'
revers['IA-MO'] = 'Des Moines'
revers['IA-NE'] = 'Missouri'
revers['IA-SD'] = 'Big Sioux'
revers['IA-WI'] = 'Mississippi'
revers['ID-OR'] = 'Snake'
revers['ID-WA'] = 'Snake'
revers['IL-IA'] = 'Mississippi'
revers['IL-IN'] = 'Wabash'
revers['IL-KY'] = 'Ohio'
revers['IL-MO'] = 'Mississippi'
revers['IN-KY'] = 'Ohio'
revers['KS-MO'] = 'Missouri'
revers['KY-MO'] = 'Mississippi'
revers['KY-WV'] = 'Big Sandy'
revers['LA-MS'] = 'Mississippi'
revers['LA-TX'] = 'Sabine'
revers['MA-RI'] = 'Palmer'
revers['MD-VA'] = 'Potomac'
revers['MD-WV'] = 'Potomac'
revers['ME-NH'] = 'Piscataqua'
revers['MI-WI'] = 'Menominee'
revers['MN-ND'] = 'Red (North)'
revers['MN-SD'] = 'Bois de Sioux'
revers['MN-WI'] = 'Mississippi'
revers['MO-NE'] = 'Missouri'
revers['MO-TN'] = 'Mississippi'
revers['NC-SC'] = 'Catawba'
revers['NC-VA'] = 'Blackwater'
revers['NE-SD'] = 'Missouri'
revers['NH-VT'] = 'Connecticut'
revers['NJ-NY'] = 'Hudson'
revers['NJ-PA'] = 'Delaware'
revers['NY-PA'] = 'Delaware'
revers['NY-VT'] = 'Poultney'
revers['OH-KY'] = 'Ohio'
revers['OH-WV'] = 'Ohio'
revers['OK-TX'] = 'Red (South)'
revers['OR-WA'] = 'Columbia'
revers['VA-WV'] = 'Potomac'

In [35]:
def dict_mapper(bord):
    if bord in revers.keys():
        return revers[bord]
    else:
        return 'N/A'

In [36]:
df['River'] = df['ST1ST2'].map(dict_mapper)

In [37]:
df.sample(20)

,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone,River
1,AL-GA,291.3,27,True,Central-Eastern,Chattahoochee
10,AZ-CA,198.4,48,True,Mountain-Pacific,Colorado
51,KS-MO,307.5,22,True,Central,Missouri
99,OR-WA,385.8,9,True,Pacific,Columbia
37,ID-MT,569.8,3,True,Mountain & Mountain-Pacific,N/A
86,NE-WY,138.0,70,True,Mountain,N/A
6,AR-MS,282.8,29,False,Central,Mississippi
39,ID-OR,305.8,24,True,Mountain & Pacific & Mountain-Pacific,Snake
35,IA-SD,85.6,85,True,Central,Big Sioux
49,IN-MI,108.4,79,True,Eastern & Central-Eastern,N/A


In [38]:
conn = sqlite3.connect('borders.db')

In [39]:
# df.to_sql('borders', conn)

In [40]:
pd.read_sql(
    """
    SELECT *
    FROM borders
    """
, conn)

,index,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone,River
0,0,AL-FL,196.1,50,1,Central,Perdido
1,1,AL-GA,291.3,27,1,Central-Eastern,Chattahoochee
2,2,AL-MS,323.9,20,1,Central,Tennessee
3,3,AL-TN,146.7,68,1,Central,N/A
4,4,AR-LA,166.5,60,1,Central,N/A
...,...,...,...,...,...,...,...
100,100,PA-WV,118.6,77,1,Eastern,N/A
101,101,SD-WY,137.7,71,1,Mountain,N/A
102,102,TN-VA,110.7,78,1,Eastern,N/A
103,103,UT-WY,173.1,58,1,Mountain,N/A


In [41]:
pd.read_sql(
    """
    SELECT *
    FROM borders
    ORDER BY LengthRank
    LIMIT 20
    """, conn
)

,index,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone,River
0,98,OK-TX,715.4,1,1,Central,Red (South)
1,14,CA-NV,608.2,2,1,Pacific,N/A
2,37,ID-MT,569.8,3,1,Mountain & Mountain-Pacific,N/A
3,91,NM-TX,536.2,4,1,Mountain & Central-Mountain,N/A
4,59,LA-MS,456.6,5,1,Central,Mississippi
5,53,KS-OK,409.1,6,1,Central,N/A
6,85,NE-SD,399.4,7,0,Central & Mountain,Missouri
7,11,AZ-NM,391.0,8,1,Mountain,N/A
8,99,OR-WA,385.8,9,1,Pacific,Columbia
9,104,VA-WV,381.2,10,1,Eastern,Potomac


In [42]:
pd.read_sql(
    """
    SELECT *
    FROM borders
    WHERE ST1ST2 LIKE '%ID%'
    ORDER BY LengthRank
    """, conn
)

,index,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone,River
0,37,ID-MT,569.8,3,1,Mountain & Mountain-Pacific,N/A
1,39,ID-OR,305.8,24,1,Mountain & Pacific & Mountain-Pacific,Snake
2,41,ID-WA,210.3,44,1,Pacific,Snake
3,42,ID-WY,170.7,59,0,Mountain,N/A
4,40,ID-UT,153.6,63,1,Mountain,N/A
5,38,ID-NV,153.0,64,0,Mountain-Pacific,N/A


In [43]:
pd.read_sql(
    """
    SELECT *
    FROM borders
    WHERE ST1ST2 LIKE '%OH%'
    ORDER BY LengthRank
    """, conn
)

,index,ST1ST2,LENGTH,LengthRank,Interstate,TimeZone,River
0,97,OH-WV,243.5,37,1,Eastern,Ohio
1,50,IN-OH,178.8,55,1,Eastern,N/A
2,55,KY-OH,155.5,62,1,Eastern,N/A
3,96,OH-PA,92.4,81,1,Eastern,N/A
4,69,MI-OH,69.7,90,1,Eastern,N/A
